### Načítanie knižníc

In [9]:
# nacitanie kniznic pre analyzu dat
import pandas as pd
import numpy as np

# nacitanie kniznic pre rozhodovacie stromy
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

# kniznica na zobrazenie frame-u
from IPython.display import IFrame

#knižnice pre znázornenie rozhodovacieho stromu
import pydotplus
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
from IPython.display import Image

### Načítanie datasetu

In [10]:
# dataset sa nachadza na tejto adrese: https://www.kaggle.com/bitsofishan/covid19-patient-symptoms
IFrame('https://www.kaggle.com', width = 800, height = 450)

# u mna je lokalizovany lokalne v projekte
# nacitam jeho cestuip
csv_file_path = "../db/COVID-19-Survey.csv"


In [11]:
# nazvy stlpcov, s ktorymi budeme pracovat
# col_names = ['age', 'gender','body-temperature', 'dry-cough', 'sour-throat', 'breathing-problem', 'wearness', 'drowsiness', 'pain-in-chest', 'diabetes','change-in-sns','travel-history']

# nacitame dataset
covid_csv = pd.read_csv(csv_file_path, sep=";")

In [12]:
covid_csv.head(5)

,age,gender,body-temperature,dry-cough,sour-throat,breathing-problem,pain-in-chest,weakness,drowsiness,diabetes,change-in-sns,travel-history,result
0,20,F,37.0,0,0,0,0,0,0,0,0,0,0
1,19,F,37.2,1,0,0,0,0,0,0,0,0,0
2,55,M,38.9,1,1,1,1,1,1,0,1,0,1
3,40,M,37.8,0,0,0,1,0,1,1,1,1,2
4,33,F,37.3,0,1,1,0,0,0,0,1,1,2


In [13]:
covid_csv.describe()

,age,body-temperature,dry-cough,sour-throat,breathing-problem,pain-in-chest,weakness,drowsiness,diabetes,change-in-sns,travel-history,result
count,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000
mean,36.456693,37.794488,0.488189,0.551181,0.433071,0.370079,0.409449,0.425197,0.244094,0.629921,0.464567,0.937008
std,14.052484,0.870532,0.501840,0.499343,0.497463,0.484738,0.493680,0.496331,0.431250,0.484738,0.500718,0.675513
min,18.000000,35.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.500000,37.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,37.600000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,45.000000,38.550000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,80.000000,39.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [14]:
# v datasete sa nachadza okrem ciselnych aj jeden stlpec s kategorialnou premennou
# get dummies prehodi kategorialne premenne na ciselne
new_covid_csv = pd.get_dummies(covid_csv, columns=["gender"], drop_first=False)

In [15]:
new_covid_csv


,age,body-temperature,dry-cough,sour-throat,breathing-problem,pain-in-chest,weakness,drowsiness,diabetes,change-in-sns,travel-history,result,gender_F,gender_M
0,20,37.0,0,0,0,0,0,0,0,0,0,0,1,0
1,19,37.2,1,0,0,0,0,0,0,0,0,0,1,0
2,55,38.9,1,1,1,1,1,1,0,1,0,1,0,1
3,40,37.8,0,0,0,1,0,1,1,1,1,2,0,1
4,33,37.3,0,1,1,0,0,0,0,1,1,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,37,38.9,1,1,1,0,1,0,0,1,0,0,1,0
123,35,37.8,0,0,0,1,0,1,1,1,1,1,0,1
124,22,37.3,0,0,1,0,0,0,0,1,0,1,1,0
125,29,38.9,0,1,1,1,1,1,0,1,0,0,0,1


In [16]:
# rozdelenie do trénovacej a testovacej množiny v pomere 80 : 20
X = new_covid_csv[['body-temperature', 'dry-cough', 'sour-throat', 'breathing-problem', 'weakness', 'drowsiness', 'pain-in-chest', 'diabetes','change-in-sns','travel-history']]
y = new_covid_csv['result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
clf = DecisionTreeClassifier()

In [18]:
# Alt+ Shift + B
clf = clf.fit(X_train,y_train)

In [19]:
y_pred = clf.predict(X_test)

In [20]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5


In [21]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[5 4 0]
 [3 5 2]
 [1 3 3]]


In [22]:

# rozhodovaci strom

import numpy as np
from sklearn.tree import _tree

def tree_to_code(tree, feature_names, Y):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    pathto=dict()

    global k
    k = 0
    def recurse(node, depth, parent):
        global k
        indent = "  " * depth

        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            s= "{} <= {} ".format( name, threshold, node )
            if node == 0:
                pathto[node]=s
            else:
                pathto[node]=pathto[parent]+' & ' +s

            recurse(tree_.children_left[node], depth + 1, node)
            s="{} > {}".format( name, threshold)
            if node == 0:
                pathto[node]=s
            else:
                pathto[node]=pathto[parent]+' & ' +s
            recurse(tree_.children_right[node], depth + 1, node)
        else:
            k=k+1
            print(k,')',pathto[parent], tree_.value[node])
    recurse(0, 1, 0)


In [23]:
tree_to_code(clf, tagy, y)

NameError: name 'tagy' is not defined